In [1]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(keras.__version__)

2.3.0
2.4.0


In [22]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Get the housing dataset
housing = fetch_california_housing()

# Split the dataset into training and testing
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

# Split the train full set into validation and training set
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [23]:
# Scaling the inputs using standard scaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [24]:
# Since the dataset is noisy, we just use a single layer with fewer neurons to avoid overfitting
# No activation needed for output layer as it is a regression problem
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

model.compile(loss='mean_squared_error', optimizer='sgd')

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                270       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [6]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
363/363 [==============================] - 2s 5ms/step - loss: 1.0795 - val_loss: 0.6214
Epoch 2/20
363/363 [==============================] - 1s 4ms/step - loss: 0.5518 - val_loss: 0.4957
Epoch 3/20
363/363 [==============================] - 2s 5ms/step - loss: 0.4949 - val_loss: 0.4861
Epoch 4/20
363/363 [==============================] - 2s 5ms/step - loss: 0.4680 - val_loss: 0.4458
Epoch 5/20
363/363 [==============================] - 2s 5ms/step - loss: 0.4563 - val_loss: 0.4354
Epoch 6/20
363/363 [==============================] - 2s 5ms/step - loss: 0.4419 - val_loss: 0.4304
Epoch 7/20
363/363 [==============================] - 1s 4ms/step - loss: 0.4328 - val_loss: 0.4176
Epoch 8/20
363/363 [==============================] - 1s 4ms/step - loss: 0.4248 - val_loss: 0.4201
Epoch 9/20
363/363 [==============================] - 1s 4ms/step - loss: 0.4187 - val_loss: 0.4202
Epoch 10/20
363/363 [==============================] - 2s 4ms/step - loss: 0.4146 - val_loss: 0.4035

In [7]:
# Evaluate the model on test dataset
mse_test = model.evaluate(X_test, y_test)

162/162 [==============================] - 0s 2ms/step - loss: 0.3790


In [8]:
# Predicting new values using model.predict
X_new = X_test[:3]
y_pred = model.predict(X_new)
print(y_pred)
print(y_test[:3])

[[4.102314 ]
 [1.3161762]
 [1.7214413]]
[5.00001 1.      1.     ]


In [9]:
# Building a non sequential model with Keras using the functional api
# Wide and Deep neural network
# It connects all or part of the inputs directly to the output layer
# This makes it possible for the network to learn both deep patterns and simple rules using the short path
# The short path can also be used to provide manually engineered features to the network
# Simple patterns in the data might be distorted by sequence of transformations
# Something like simple binary variables like [0, 1] might be sent through the short path

# This layer defines what input we can get for the model, shape and dtypes
input_ = keras.layers.Input(shape=X_train.shape[1:])

# As soon as hidden 1 is created, we call it like a function (hence functional API) passing the input to that layer
hidden1 = keras.layers.Dense(30, activation='relu')(input_)

# hidden 1 connects to hidden 2
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)

# Concat layers concats the inputs from input layer and the hidden 2 layer into 1 layer
concat = keras.layers.Concatenate()([input_, hidden2])

# Output layer takes the inputs from the concat layer
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

In [10]:
# What if we want to send one subset of features through the deep path and one subset through the shallow path
# We can use the functional API for this as well
input_A = keras.layers.Input(shape=[5], name='wide_input')
input_B = keras.layers.Input(shape=[6], name='deep_input')
hidden1 = keras.layers.Dense(30, activation='relu')(input_B)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name='output')(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])

# When we call the fit model method, we must pass a pair of matrices like (input_A, input_B)
# Alternatively, we can pass values as dictionary {'wide_input': X_train_A, 'deep_input': X_train_B}

In [11]:
model.compile(loss='mse', optimizer=keras.optimizers.SGD(lr=1e-3))

In [12]:
# Features 0 to 4 passed to wide and 2 to 7 passed to deep
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]


In [13]:
# Fitting the model
history = model.fit((X_train_A, X_train_B), y_train, epochs=20, validation_data=((X_valid_A, X_valid_B), y_valid))

Epoch 1/20
363/363 [==============================] - 2s 5ms/step - loss: 1.8806 - val_loss: 0.9521
Epoch 2/20
363/363 [==============================] - 2s 5ms/step - loss: 0.8254 - val_loss: 0.7325
Epoch 3/20
363/363 [==============================] - 2s 5ms/step - loss: 0.7277 - val_loss: 0.6748
Epoch 4/20
363/363 [==============================] - 1s 4ms/step - loss: 0.6789 - val_loss: 0.6348
Epoch 5/20
363/363 [==============================] - 1s 4ms/step - loss: 0.6426 - val_loss: 0.6040
Epoch 6/20
363/363 [==============================] - 2s 4ms/step - loss: 0.6131 - val_loss: 0.5789
Epoch 7/20
363/363 [==============================] - 2s 4ms/step - loss: 0.5883 - val_loss: 0.5581
Epoch 8/20
363/363 [==============================] - 2s 5ms/step - loss: 0.5676 - val_loss: 0.5405
Epoch 9/20
363/363 [==============================] - 2s 4ms/step - loss: 0.5507 - val_loss: 0.5267
Epoch 10/20
363/363 [==============================] - 2s 5ms/step - loss: 0.5368 - val_loss: 0.5154

In [14]:
mse_test = model.evaluate((X_test_A, X_test_B), y_test)


162/162 [==============================] - 0s 2ms/step - loss: 0.4893


In [15]:
y_pred = model.predict((X_new_A, X_new_B))
print(y_pred)
print(y_test[:3])


[[3.9354904]
 [1.5702002]
 [1.7777386]]
[5.00001 1.      1.     ]


In [ ]:
# It is possible to have multiple outputs
# For example, locate and classify main object in the picture. This is both a regression task and classification task
# Auxiliary outputs can be made to make sure network is learning something

# To add extra outputs, just connect them to appropriate layers and add them to your models list of outputs
# Example
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

# Each output will needs its own loss function
model.compile(loss=['mse', 'mse'], loss_weights=[0.9, 0.1], optimizer='sgd')
history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=20, validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))

total_loss, main_loss, aux_loss = model.evaluate([X_test_A, X_test_B], [y_test, y_test])
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])

In [16]:
# Subclassing API
# Create the layers in the constructor
# And do the computations in the call method

class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        super().__init__(**kwargs) # handles the standard args (e.g. name)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
    
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

model = WideAndDeepModel()

# Saving and loading the model
model.save('my_keras_model.h5')
model = keras.model.load_model('my_keras_model.h5')


In [ ]:
# To save states of model during training at different checkpoints, you can use callbacks to save them
# The fit method accepts arguments that lets you specifiy objects it calls before start and end of training
# Start and end of each epoch
# Stat and end of processing each batch

# By default saves state and end of each epoch
# Save best only just saves the model with best performance on the validation set
checkpoint_cb = keras.callbacks.ModelCheckpoint('my_keras.model.h5', save_best_only=True)
history = model.fit(X_train, y_train, epochs=10, callbacks=[checkpoint_cb])

# patience argument is to check how many epochs to check if the performance does not improve
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)


In [ ]:
# Implementing your own callback
class PrintValTrainRatioCallBack(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print(logs['val_loss']/logs['loss'])

In [25]:
# Tensorboard
# To use tensorboard you have to output special binary logs call event files
# Each binary record is called summary

# The tensorboard server reads log files from directory and automatically updates the visualizations based on these
import os
root_logdir = os.path.join(os.curdir, 'my_logs')

def get_run_logdir():
    import time
    run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
print(run_logdir)

.\my_logs\run_2020_09_12-16_06_59


In [26]:
# Callback for tensorboard
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_cb])

Epoch 1/30
  1/363 [..............................] - ETA: 0s - loss: 3.8327WARNING:tensorflow:From C:\ProgramData\Anaconda3\envs\deep-learning\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
363/363 [==============================] - 3s 7ms/step - loss: 0.9976 - val_loss: 0.6717
Epoch 2/30
363/363 [==============================] - 2s 4ms/step - loss: 0.7764 - val_loss: 0.4954
Epoch 3/30
363/363 [==============================] - 2s 4ms/step - loss: 0.4695 - val_loss: 0.4800
Epoch 4/30
363/363 [==============================] - 1s 4ms/step - loss: 0.4517 - val_loss: 0.4657
Epoch 5/30
363/363 [==============================] - 1s 4ms/step - loss: 0.4415 - val_loss: 0.4552
Epoch 6/30
363/363 [==============================] - 1s 4ms/step - loss: 0.4293 - val_loss: 0.4483
Epoch 7/30
363/363 [==============

In [ ]:
# Hyperparameter tuning 
# One option is wrap it like a GridSearchCV or RandomizedSearchCV in sklearn

def build_model(n_hidden=1, n_nuerons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_nuerons, activation='relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

In [27]:
# Keras regressor is a thin wrapper around build model and we can use it like any model in scikit  learn
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)
keras_reg.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=10)])
mse_test = keras_reg.score(X_test, y_test)
y_pred = keras.reg.predict(X_new)

SyntaxError: invalid syntax (<ipython-input-27-51fc561ecd36>, line 3)

In [28]:
from scipy.stats import reciprocal
import numpy as np
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    'n_hidden': [0, 1, 2, 3],
    'n_nuerons': np.arange(1, 100),
    'learning_rate': reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=10)])